In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


df = pd.read_csv("../../data/table.csv", delimiter=';')


df["Tid(norsk normaltid)"] = pd.to_datetime(df["Tid(norsk normaltid)"], format='%d.%m.%Y %H:%M', errors='coerce')

df["Lufttemperatur"] = pd.to_numeric(df["Lufttemperatur"].astype(str).str.replace(',', '.'), errors='coerce')

df = df.sort_values("Tid(norsk normaltid)")

# Step 6: Plot the data
plt.figure(figsize=(10, 5))
plt.plot(df["Tid(norsk normaltid)"], df["Lufttemperatur"], label="Temperatur", color='b')
plt.xlabel("Tid")
plt.ylabel("Temperatur (°C)")
plt.title("Temperatur over tid (Mars 2024 - Mars 2025)")
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()  # Adjust layout to prevent label cutoff
plt.show()

ModuleNotFoundError: No module named 'pandas'